# Binary Classification with a Bank Churn Dataset
## Playground Series - Season 4, Episode 1
### Dataset Description
The dataset for this competition (both train and test) was generated from a deep learning model trained on the Bank Customer Churn Prediction dataset. Feature distributions are close to, but not exactly the same, as the original. Feel free to use the original dataset as part of this competition, both to explore differences as well as to see whether incorporating the original in training improves model performance.
### About Dataset
- Customer ID: A unique identifier for each customer
- Surname: The customer's surname or last name
- Credit Score: A numerical value representing the customer's credit score
- Geography: The country where the customer resides (France, Spain or Germany)
- Gender: The customer's gender (Male or Female)
- Age: The customer's age.
- Tenure: The number of years the customer has been with the bank
- Balance: The customer's account balance
- NumOfProducts: The number of bank products the customer uses (e.g., savings account, credit card)
- HasCrCard: Whether the customer has a credit card (1 = yes, 0 = no)
- IsActiveMember: Whether the customer is an active member (1 = yes, 0 = no)
- EstimatedSalary: The estimated salary of the customer
- Exited: Whether the customer has churned (1 = yes, 0 = no)
### Files
- train.csv - the training dataset; Exited is the binary target
- test.csv - the test dataset; your objective is to predict the probability of Exited
- sample_submission.csv - a sample submission file in the correct format
### Evaluation
Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.
### Your Goal:
For this Episode of the Series, your task is to predict whether a customer continues with their account or closes it (e.g., churns). Good luck!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import missingno as msno

from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import warnings
warnings.filterwarnings("ignore")

### Credits
Override from original dataset
@paddykb 
- https://www.kaggle.com/competitions/playground-series-s4e1/discussion/469859
- https://www.kaggle.com/competitions/playground-series-s4e1/discussion/464886#2583413

lets see if this does help later

------------

### Import Data + Pre-check Data Structure

In [12]:
# Importing all data
train_data = pd.read_csv(r'data\train.csv')
test_data = pd.read_csv(r'data\test.csv')
original_data = pd.read_csv(r'data\Churn_Modelling.csv')
sample_submission = pd.read_csv(r'data\sample_submission.csv')

In [13]:
print(train_data.columns)
train_data.head()

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [14]:
print(test_data.columns)
test_data.head()

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [15]:
print(original_data.columns)
original_data.head()

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


In [22]:
train_data.shape , test_data.shape,original_data.shape

((165034, 14), (110023, 13), (10002, 14))

Interpreting the data

1. We can see that original_data has 'RowNumber' instead of 'id' as column name
2. Both train_data and original_data has 'Exited' (need to split)

In [ ]:
# Rename column name
original_data.rename(columns = {'RowNumber': 'id'}, inplace=True)

For the simple visual table for:
 - missing values
 - unique values
 - data type
 
Credit: @tarundirector

https://www.kaggle.com/code/tarundirector/binary-classification-bank-churn-eda

In [38]:
# Creating a table for missing values, unique values and data types of the features

missing_values_train = pd.DataFrame({'Feature': train_data.columns,
                              '[TRAIN] No. of Missing Values': train_data.isnull().sum().values,
                              '[TRAIN] % of Missing Values': ((train_data.isnull().sum().values)/len(train_data)*100)})

missing_values_test = pd.DataFrame({'Feature': test_data.columns,
                             '[TEST] No.of Missing Values': test_data.isnull().sum().values,
                             '[TEST] % of Missing Values': ((test_data.isnull().sum().values)/len(test_data)*100)})

missing_values_original = pd.DataFrame({'Feature': original_data.columns,
                             '[ORIGINAL] No.of Missing Values': original_data.isnull().sum().values,
                             '[ORIGINAL] % of Missing Values': ((original_data.isnull().sum().values)/len(original_data)*100)})

unique_values = pd.DataFrame({'Feature': train_data.columns,
                              'No. of Unique Values[FROM TRAIN]': train_data.nunique().values})

feature_types = pd.DataFrame({'Feature': train_data.columns,
                              'DataType': train_data.dtypes})

merged_df = pd.merge(missing_values_train, missing_values_test, on='Feature', how='left')
merged_df = pd.merge(merged_df, missing_values_original, on='Feature', how='left')
merged_df = pd.merge(merged_df, unique_values, on='Feature', how='left')
merged_df = pd.merge(merged_df, feature_types, on='Feature', how='left')

# add gradient to see missing values
cm = sns.light_palette("red", as_cmap=True)
merged_df.style.text_gradient(cmap=cm)

,Feature,[TRAIN] No. of Missing Values,[TRAIN] % of Missing Values,[TEST] No.of Missing Values,[TEST] % of Missing Values,[ORIGINAL] No.of Missing Values,[ORIGINAL] % of Missing Values,No. of Unique Values[FROM TRAIN],DataType
0,id,0,0.000000,0.000000,0.000000,nan,nan,165034,int64
1,CustomerId,0,0.000000,0.000000,0.000000,0.000000,0.000000,23221,int64
2,Surname,0,0.000000,0.000000,0.000000,0.000000,0.000000,2797,object
3,CreditScore,0,0.000000,0.000000,0.000000,0.000000,0.000000,457,int64
4,Geography,0,0.000000,0.000000,0.000000,1.000000,0.009998,3,object
5,Gender,0,0.000000,0.000000,0.000000,0.000000,0.000000,2,object
6,Age,0,0.000000,0.000000,0.000000,1.000000,0.009998,71,float64
7,Tenure,0,0.000000,0.000000,0.000000,0.000000,0.000000,11,int64
8,Balance,0,0.000000,0.000000,0.000000,0.000000,0.000000,30075,float64
9,NumOfProducts,0,0.000000,0.000000,0.000000,0.000000,0.000000,4,int64


### Split Numerical & Categorical Features

In [19]:
train_data.info()
train_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.0000,1.650340e+05,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,82516.5000,1.569201e+07,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,47641.3565,7.139782e+04,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,0.0000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,41258.2500,1.563314e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,82516.5000,1.569017e+07,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,123774.7500,1.575682e+07,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,165033.0000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [21]:
numerical_cols = train_data.select_dtypes(include=[np.number])
categorical_cols = train_data.select_dtypes(exclude=[np.number])
numerical_cols.head(), categorical_cols.head()

(   id  CustomerId  CreditScore   Age  Tenure    Balance  NumOfProducts  \
 0   0    15674932          668  33.0       3       0.00              2   
 1   1    15749177          627  33.0       1       0.00              2   
 2   2    15694510          678  40.0      10       0.00              2   
 3   3    15741417          581  34.0       2  148882.54              1   
 4   4    15766172          716  33.0       5       0.00              2   
 
    HasCrCard  IsActiveMember  EstimatedSalary  Exited  
 0        1.0             0.0        181449.97       0  
 1        1.0             1.0         49503.50       0  
 2        1.0             0.0        184866.69       0  
 3        1.0             1.0         84560.88       0  
 4        1.0             1.0         15068.83       0  ,
           Surname Geography Gender
 0  Okwudilichukwu    France   Male
 1   Okwudiliolisa    France   Male
 2           Hsueh    France   Male
 3             Kao    France   Male
 4       Chiemenam     Sp